# Lab | Revisiting Machine Learning Case Study

- In this lab, you will use `learningSet.csv` file which we used during the class. 

## Instructions

Complete the following steps on the categorical columns in the dataset:

- Check for null values in all the columns
- Create a new empty list called `drop_list`. We will append to this list a set of columns to be dropped later. Add the following columns to this:
    - `OSOURCE` - symbol definitions not provided, too many categories
    - `ZIP` - we are including states already
- Identify columns that have over 85% missing values and add them to the previous list.
- Remove the columns included in the `drop_list` from the dataframe
- Now, reduce the number of categories in the column `GENDER`. The column should only have either "M" for males, "F" for females, and "other" for all the rest
    - Note that there are a few null values in the column. We will first replace those null values using the code below:

    ```python
    print(categorical['GENDER'].value_counts())
    categorical['GENDER'] = categorical['GENDER'].fillna('F')
    ```


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [4]:
data = pd.read_csv('~/Desktop/bootcamp/Week_7/Day_1/Morning/learningSet.csv')

/var/folders/sf/w99qq72n7ld5cplb4kcgbmxm0000gn/T/ipykernel_23541/3878165107.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('~/Desktop/bootcamp/Week_7/Day_1/Morning/learningSet.csv')


In [5]:
data.shape

(95412, 481)

In [12]:
# check full values in all columns]
nulls_percent_df = pd.DataFrame()
nulls_percent_df = pd.DataFrame(data.isna().sum().reset_index())
nulls_percent_df

index    0
0     ODATEDW    0
1     OSOURCE    0
2       TCODE    0
3       STATE    0
4         ZIP    0
..        ...  ...
476  MDMAUD_R    0
477  MDMAUD_F    0
478  MDMAUD_A    0
479  CLUSTER2  132
480  GEOCODE2  132

[481 rows x 2 columns]

In [63]:
# create empy list
drop_list = []
drop_list.extend(['OSOURCE', 'ZIP'])
#drop_list.append('OSOURCE')
#drop_list.append('ZIP')

In [28]:
drop_list

['OSOURCE', 'ZIP']

In [44]:
# Define the function here
def columns_above_threshold(df: pd.DataFrame, threshold=0.25) -> list:
    '''
    this function takes as input a dataframe and a threshold to return a list of columns
    '''
    df_copy = df.copy()
    print('reading data...')
    column_list_nas = pd.DataFrame(df_copy.isna().sum()/len(df_copy)).reset_index() # obtaining the % of NAs per column
    column_list_nas.columns = ['column_name', 'nulls_percentage']
    print('creating new dataframe with na %s') 
    
    col_list = column_list_nas[column_list_nas['nulls_percentage']>threshold]['column_name'].tolist() # filtering the columns with NAs above the threshold
    print('creating filtered list...')
    
    #col_list = col_list['column_name'].tolist() # filtering the columns with NAs above the threshold
    print(len(col_list))
    
    return col_list

In [45]:
# call function
above_85_list = columns_above_threshold(data, 0.85)

reading data...
creating new dataframe with na %s
creating filtered list...
25


In [ ]:
# option 1 to append new values from list to previous list
for col in above_85_list:
    if col in drop_list:
        pass
    elif col not in drop_list:
        drop_list.append(col)
    else:
        pass
len(drop_list)

In [73]:
# option 2 to append new values from list to previous list
drop_list += [col for col in above_85_list if col not in drop_list]
len(drop_list)

27

In [74]:
# option 3 to append new values from list to previous list

drop_list = [col for col in above_85_list if col not in drop_list] + drop_list
len(drop_list)

27

In [76]:
def remove_cols_from_df(df:pd.DataFrame, remove_list) -> pd.DataFrame:
    '''
    this fn removes columns list from a dataframe
    input:df and list
    output: df
    '''
    df_copy = df.copy()
    dataframe_shape = df_copy.shape
    dataframe_col_num = int(len(df_copy.columns))
    print(f'reading data... dataframe_shape: {dataframe_shape}')
    
    df_copy.drop(columns=remove_list, inplace=True, errors='ignore')
    dataframe_col_num_post = int(len(df_copy.columns))
    removed_columns_count = int(dataframe_col_num_post - dataframe_col_num)
    dataframe_shape_post = df_copy.shape
    print(f'removing {removed_columns_count} columns... post_dataframe_shape: {dataframe_shape_post}')
    return df_copy
    


In [77]:
df_test = remove_cols_from_df(data, drop_list)

reading data... dataframe_shape: (95412, 481)
removing -27 columns... post_dataframe_shape: (95412, 454)


In [80]:
df_test.shape

(95412, 454)

In [81]:
data = df_test

## X y Split and num / cat split

In [82]:
Y = data[['TARGET_B', 'TARGET_D']]
Y.head()

TARGET_B  TARGET_D
0         0       0.0
1         0       0.0
2         0       0.0
3         0       0.0
4         0       0.0

In [83]:
numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])
numerical.head()

ODATEDW  TCODE   DOB   AGE  INCOME  WEALTH1  HIT  MBCRAFT  MBGARDEN  \
0     8901      0  3712  60.0     NaN      NaN    0      NaN       NaN   
1     9401      1  5202  46.0     6.0      9.0   16      0.0       0.0   
2     9001      1     0   NaN     3.0      1.0    2      0.0       0.0   
3     8701      0  2801  70.0     1.0      4.0    2      0.0       0.0   
4     8601      0  2001  78.0     3.0      2.0   60      1.0       0.0   

   MBBOOKS  MBCOLECT  MAGFAML  MAGFEM  MAGMALE  PUBGARDN  PUBCULIN  PUBHLTH  \
0      NaN       NaN      NaN     NaN      NaN       NaN       NaN      NaN   
1      3.0       1.0      1.0     1.0      0.0       0.0       0.0      2.0   
2      1.0       0.0      0.0     0.0      0.0       0.0       0.0      1.0   
3      0.0       0.0      0.0     0.0      0.0       0.0       0.0      0.0   
4      9.0       0.0      4.0     1.0      0.0       0.0       0.0      4.0   

   PUBDOITY  PUBNEWFN  PUBPHOTO  PUBOPP  MALEMILI  MALEVET  VIETVETS  \
0       NaN       NaN       NaN     NaN         0       39        34   
1       0.0       3.0       0.0     0.0         0       15        55   
2       0.0       0.0       0.0     0.0         0       20        29   
3       0.0       0.0       0.0     1.0         0       23        14   
4       0.0       1.0       0.0     1.0         1       28         9   

   WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  WEALTH2  POP901  POP902  POP903  \
0        18        10         2       1      5.0     992     264     332   
1        11         6         2       1      9.0    3611     940     998   
2        33         6         8       1      1.0    7001    2040    2669   
3        31         3         0       3      0.0     640     160     219   
4        53        26         3       2      NaN    2520     627     761   

   POP90C1  POP90C2  POP90C3  POP90C4  POP90C5  ETH1  ETH2  ETH3  ETH4  ETH5  \
0        0       35       65       47       53    92     1     0     0    11   
1       99        0        0       50       50    67     0     0    31     6   
2        0        2       98       49       51    96     2     0     0     2   
3        0        8       92       54       46    61     0     0    11    32   
4       99        0        0       46       54     2    98     0     0     1   

   ETH6  ETH7  ETH8  ETH9  ETH10  ETH11  ETH12  ETH13  ETH14  ETH15  ETH16  \
0     0     0     0     0      0      0      0     11      0      0      0   
1     4     2     6     4     14      0      0      2      0      1      4   
2     0     0     0     0      0      0      0      2      0      0      0   
3     6     2     0     0      0      0      0     31      0      0      1   
4     0     0     0     0      0      0      0      0      0      0      0   

   AGE901  AGE902  AGE903  AGE904  AGE905  AGE906  AGE907  CHIL1  CHIL2  \
0      39      48      51      40      50      54      25     31     42   
1      34      41      43      32      42      45      32     33     46   
2      35      43      46      37      45      49      23     35     40   
3      32      40      44      34      43      47      25     45     35   
4      33      45      50      36      46      50      27     34     43   

   CHIL3  AGEC1  AGEC2  AGEC3  AGEC4  AGEC5  AGEC6  AGEC7  CHILC1  CHILC2  \
0     27     11     14     18     17     13     11     15      12      11   
1     21     13     14     33     23     10      4      2      11      16   
2     25     13     20     19     16     13     10      8      15      14   
3     20     15     25     17     17     12      7      7      20      17   
4     23     14     21     13     15     20     12      5      13      15   

   CHILC3  CHILC4  CHILC5  HHAGE1  HHAGE2  HHAGE3  HHN1  HHN2  HHN3  HHN4  \
0      34      25      18      26      10      23    18    33    49    28   
1      36      22      15      12       1       5     4    21    75    55   
2      30      22      19      25      10      23    21    35    44    22   
3      30      14    

In [84]:
categorical = data.select_dtypes([object])
categorical.head()

STATE MAILCODE PVASTATE NOEXCH RECINHSE RECP3 RECPGVG RECSWEEP MDMAUD  \
0    IL                        0                                   XXXX   
1    CA                        0                                   XXXX   
2    NC                        0                                   XXXX   
3    CA                        0                                   XXXX   
4    FL                        0        X     X                    XXXX   

  DOMAIN CLUSTER AGEFLAG HOMEOWNR CHILD03 CHILD07 CHILD12 CHILD18 GENDER  \
0     T2      36                                                       F   
1     S1      14       E        H                               M      M   
2     R2      43                U                                      M   
3     R2      44       E        U                                      F   
4     S2      16       E        H                                      F   

  DATASRCE SOLP3 SOLIH MAJOR GEOCODE COLLECT1 VETERANS BIBLE CATLG HOMEE PETS  \
0                                                                               
1        3                         2                                            
2        3                                                                      
3        3                                                                      
4        3          12                                     Y     Y              

  CDPLAY STEREO PCOWNERS PHOTO CRAFTS FISHER GARDENIN BOATS WALKER KIDSTUFF  \
0                                                                             
1                                                                             
2                                                                             
3                                                                             
4             Y                     Y               Y            Y            

  CARDS PLATES LIFESRC PEPSTRFL RFA_2 RFA_3 RFA_4 RFA_5 RFA_6 RFA_7 RFA_8  \
0                             X   L4E   S4E   S4E   S4E   S4E   S4E   S4E   
1                                 L2G   A2G   A2G   A2G   A2G   A1E   A1E   
2                             X   L4E   S4E   S4E   S4E   S4E   S4F   S4F   
3                             X   L4E   S4E   S4E   S4E   S4E   S4E   S4E   
4     Y              3            L2F   A2F   A2F   A2F   A1D   I2D   A1E   

  RFA_9 RFA_10 RFA_11 RFA_12 RFA_13 RFA_14 RFA_15 RFA_16 RFA_17 RFA_18 RFA_19  \
0   S4E    S4E    S4E    S4E    S4E    S4E    S4E    S4E    S4E    S4E    S4E   
1   A1E    A1E    A1E    A1E                         L1E                  N1E   
2   S4F           S4F    S4F    S4F    S4F    S4F    S4F           S4D    S4D   
3   S4E           S4E    S4E    S4E    S4E    S4E    S4E    S2D    S2D    A1D   
4   A1E    L1D    A1E    A1E    L1D    L3D           L3D    A2D    A2D    A3D   

  RFA_20 RFA_21 RFA_22 RFA_23 RFA_24 RFA_2R RFA_2A MDMAUD_R MDMAUD_F MDMAUD_A  \
0    S4E    S4E    S4E    S4E    S4E      L      E        X        X        X   
1    N1E    N1E    N1E           F1E      L      G        X        X        X   
2                  S4D    S4D    S3D      L      E        X        X        X   
3    A1D    A1D    A1D                    L      E        X        X        X   
4    A3D    A3D    I4E    A3D    A3D      L      F        X        X        X   

  GEOCODE2  
0        C  
1        A  
2        C  
3        C  
4        A

## Reduce gender categories

In [85]:
# reduce GENDER categories to M F and other + replace nulls
print(categorical['GENDER'].value_counts())
categorical['GENDER'] = categorical['GENDER'].fillna('F')

GENDER
F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: count, dtype: int64


In [87]:
print(categorical['GENDER'].value_counts())


GENDER
F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: count, dtype: int64


In [100]:
type(categorical['GENDER'].unique().tolist())

list

In [104]:
gender_unique = categorical['GENDER'].unique().tolist()
new_gender_options = ['F','M','other']
other_genders = [value for value in gender_unique if value not in new_gender_options]

def clean_gender(x):
    if x in other_genders:
        return 'other'
    else:
        return x
        
categorical['GENDER'] = list(map(clean_gender, categorical['GENDER']))

In [105]:
print(categorical['GENDER'].value_counts())


GENDER
F        51277
M        39094
other     5041
Name: count, dtype: int64


## Next steps